In [1]:
import torch as th
import numpy as np
from torch.optim import Adam, lr_scheduler
from torch.nn import functional as F
from tqdm import tqdm
from utils import data, metrics
import Frequentist_main as FCNN
import Bayesian_main as BCNN

In [2]:
print(th.cuda.is_available())

False


In [3]:
activation_type = 'softplus'  # 'softplus' or 'relu'
priors={
    'prior_mu': 0,
    'prior_sigma': 0.1,
    'posterior_mu_initial': (0, 0.1),  # (mean, std) normal_
    'posterior_rho_initial': (-5, 0.1),  # (mean, std) normal_
}

n_epochs = 200
lr_start = 0.001
num_workers = 4
valid_size = 0.2
batch_size = 256
train_ens = 1
valid_ens = 1
beta_type = 0.1 

In [4]:
trainset, testset, inputs, outputs = data.getDataset('CIFAR10')
train_loader, valid_loader, test_loader = data.getDataloader(
    trainset, testset, valid_size, batch_size, num_workers)
net = BBBAlexNet(outputs, inputs, priors, activation_type).to(device)

#ckpt_dir = f'checkpoints/{dataset}/bayesian'
#ckpt_name = f'checkpoints/{dataset}/bayesian/model_{net_type}_{layer_type}_{activation_type}.pt'

#if not os.path.exists(ckpt_dir):
 #   os.makedirs(ckpt_dir, exist_ok=True)

criterion = metrics.ELBO(len(trainset)).to(device)
optimizer = Adam(net.parameters(), lr=lr_start)
lr_sched = lr_scheduler.ReduceLROnPlateau(optimizer, patience=6, verbose=True)
valid_loss_max = np.Inf
for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    train_loss, train_acc, train_kl = train_model(net, optimizer, criterion, train_loader, num_ens=train_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    valid_loss, valid_acc = validate_model(net, criterion, valid_loader, num_ens=valid_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    lr_sched.step(valid_loss)

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, train_loss, train_acc, valid_loss, valid_acc, train_kl))

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)>

In [7]:
!pip install --upgrade certifi
